In [9]:
from nuscenes import NuScenes
import os
import numpy as np

nusc = NuScenes(version='v1.0-mini', dataroot='/root/main/dataset/nuscenes/', verbose=True)

array = []
for i in range(int(len(nusc.sample)*0.70), int(len(nusc.sample)*0.85)):
    base = nusc.sample[i]
    for j, f in enumerate(os.listdir('/root/main/dataset/nuscenes/samples/LIDAR_TOP')):
        if f[42:-8] == str(base['timestamp']):
            break
    sample = 'samples/LIDAR_TOP/' + f
    lidarseg = 'lidarseg/v1.0-mini/' + base['data']['LIDAR_TOP'] + '_lidarseg.bin'
    token = base['data']['LIDAR_TOP']
    array.append([sample, lidarseg, token])
list_frames = array

Loading NuScenes tables for version v1.0-mini...
Loading nuScenes-lidarseg...
32 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
404 lidarseg,
Done loading in 0.860 seconds.
Reverse indexing ...
Done reverse indexing in 0.2 seconds.


In [7]:
list_frames

[['samples/LIDAR_TOP/n015-2018-10-08-15-36-50+0800__LIDAR_TOP__1538984253447765.pcd.bin',
  'lidarseg/v1.0-mini/76c090ac27774e66a592f40998d5537c_lidarseg.bin',
  '76c090ac27774e66a592f40998d5537c'],
 ['samples/LIDAR_TOP/n015-2018-11-21-19-38-26+0800__LIDAR_TOP__1542800367947576.pcd.bin',
  'lidarseg/v1.0-mini/5e8bbadecb8640c69a12857c6992f44c_lidarseg.bin',
  '5e8bbadecb8640c69a12857c6992f44c'],
 ['samples/LIDAR_TOP/n015-2018-11-21-19-38-26+0800__LIDAR_TOP__1542800368447460.pcd.bin',
  'lidarseg/v1.0-mini/77a132cf54f84881bdc8800e82068d4a_lidarseg.bin',
  '77a132cf54f84881bdc8800e82068d4a'],
 ['samples/LIDAR_TOP/n015-2018-11-21-19-38-26+0800__LIDAR_TOP__1542800368948430.pcd.bin',
  'lidarseg/v1.0-mini/6e9f7877390a4533a6bbf711516173a1_lidarseg.bin',
  '6e9f7877390a4533a6bbf711516173a1'],
 ['samples/LIDAR_TOP/n015-2018-11-21-19-38-26+0800__LIDAR_TOP__1542800369448339.pcd.bin',
  'lidarseg/v1.0-mini/560181e1085341a3a6edfd87710dc312_lidarseg.bin',
  '560181e1085341a3a6edfd87710dc312'],
 ['sa

In [17]:
class ClassMapper:
    def __init__(self):
        current_folder = '/home/HyperLiDAR/datasets/'
        self.mapping = np.load(
            os.path.join(current_folder, "mapping_class_index_nuscenes.npy")
        )

    def get_index(self, x):
        return self.mapping[x] if x < len(self.mapping) else 0

In [18]:
pc_load = list_frames[0]
rootdir = '/root/main/dataset/nuscenes/'
mapper = np.vectorize(ClassMapper().get_index)

pc = np.fromfile(
    os.path.join(rootdir, pc_load[0]),
    dtype=np.float32,
)
pc = pc.reshape((-1, 5))[:, :4]

# Load segmentation labels
labels = np.fromfile(
    os.path.join(rootdir, pc_load[1]),
    dtype=np.uint8,
)

labels = mapper(labels)

pc, labels, pc_load[2]

(array([[-3.2174673e+00, -1.1190159e-01, -1.9096432e+00,  2.4000000e+01],
        [-3.4635863e+00, -1.0483936e-01, -1.9471916e+00,  3.0000000e+01],
        [-3.7374966e+00, -9.7581968e-02, -1.9880142e+00,  2.1000000e+01],
        ...,
        [-9.4530602e+00,  2.4750130e-02,  1.3285465e+00,  9.0000000e+00],
        [-1.0203167e+01,  3.0272897e-02,  1.6763263e+00,  5.2000000e+01],
        [-9.9567575e+00,  3.3019613e-02,  1.8759612e+00,  2.1000000e+01]],
       dtype=float32),
 array([14, 14, 14, ..., 16, 16, 16]),
 '76c090ac27774e66a592f40998d5537c')

In [24]:
np.save('/home/points.npy', pc)

In [25]:
np.save('/home/labels.npy', labels)

In [27]:
points = np.load('/home/points.npy')
points.shape

(34752, 4)

In [29]:
max(labels)

16